In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
# import tensorflow
from keras.models import load_model
from keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
# from tensorflow.keras.Model import Sequential, load_model

In [2]:
import pandas as pd

dataset_path = "static/dataset/Harga_Cabai_Jawa_2020_2024.csv"
dataframe = pd.read_csv(dataset_path)
dataframe

,Tanggal_Panen,Harga
0,2015-01-13,20365
1,2015-01-26,20794
2,2015-02-09,21224
3,2015-02-20,21112
4,2015-03-06,20649
...,...,...
277,2024-03-10,10000
278,2024-03-23,10402
279,2024-04-06,11093
280,2024-04-17,11932


## EDA

In [3]:
from datetime import datetime, timedelta

def konversi_tanggal(jumlah_hari):
    # Tanggal awal
    tanggal_awal = datetime(2015, 1, 13)

    # Kurangkan jumlah hari
    tanggal_hasil = tanggal_awal - timedelta(days=jumlah_hari)

    return tanggal_hasil

In [6]:

# Select the first 660 rows
new_df = dataframe.iloc[:660].copy()

# Add a new column named 'tanggal' with converted index
new_df.insert(0, "tanggal", new_df["Tanggal_Panen"])
new_df["tanggal"] = pd.to_datetime(new_df["tanggal"])

# Drop the 'Tanggal_Panen' column
new_df.drop(columns=["Tanggal_Panen"], inplace=True)

# Display the updated DataFrame
print(new_df)



       tanggal  Harga 
0   2015-01-13   20365
1   2015-01-26   20794
2   2015-02-09   21224
3   2015-02-20   21112
4   2015-03-06   20649
..         ...     ...
277 2024-03-10   10000
278 2024-03-23   10402
279 2024-04-06   11093
280 2024-04-17   11932
281 2024-04-28   11961

[282 rows x 2 columns]


## MODELLING

In [11]:
# Fungsi untuk membuat dataset time series
def create_dataset(dataset, time_steps=1):
    X, y = [], []
    for i in range(len(dataset) - time_steps):
        X.append(dataset[i:(i + time_steps), 0])
        y.append(dataset[i + time_steps, 0])
    return np.array(X), np.array(y)

In [10]:
# Preprocessing data
def preprocess_data(df):
    dataset = df['harga'].values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)
    return dataset, scaler

In [9]:
# Split data into train and test sets
def split_data(data):
    train_size = int(len(data) * 0.7)
    test_size = len(data) - train_size
    train, test = data[0:train_size,:], data[train_size:len(data),:]
    return train, test

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def build_model(time_steps, features):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(time_steps, features)),
        LSTM(50, return_sequences=False),
        Dense(25),
        Dense(1)
    ])
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    return model

# Define jumlah time steps dan fitur
time_steps = 30  # Ganti sesuai kebutuhan Anda
features = 1     # Jumlah fitur dalam data, dalam kasus ini hanya harga

# Buat model
model = build_model(time_steps, features)


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def preprocess_data(df):
    # Convert the 'Harga' column to numpy array and reshape for scaler
    dataset = df['Harga'].values.reshape(-1, 1)
    
    # Initialize and fit the MinMaxScaler
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)
    
    return dataset, scaler

# Load dataset
path_to_dataset = 'static/dataset/Harga_Cabai_Jawa_2020_2024.csv'  
df = pd.read_csv(path_to_dataset)

# Preprocess data
dataset, scaler = preprocess_data(df)


In [ ]:
# Split data
train, test = split_data(dataset)

# Create dataset for LSTM
time_steps = 30
X_train, y_train = create_dataset(train, time_steps)
X_test, y_test = create_dataset(test, time_steps)

# Reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [9]:
 # Build LSTM model
model = build_model(time_steps, 1)

# Train model
model.fit(X_train, y_train, epochs=100, batch_size=32)

X_test or y_test is empty. Check the dataset or time_steps.


In [ ]:
# Evaluate model
train_loss = model.evaluate(X_train, y_train, verbose=0)
print('Train Loss:', train_loss)
test_loss = model.evaluate(X_test, y_test, verbose=0)
print('Test Loss:', test_loss)


In [ ]:
# Predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Inverse transform predictions
train_predict = scaler.inverse_transform(train_predict)
y_train = scaler.inverse_transform([y_train])
test_predict = scaler.inverse_transform(test_predict)
y_test = scaler.inverse_transform([y_test])


In [ ]:
# Plotting
plt.plot(np.arange(0, len(y_train[0])), y_train[0], label='Actual Train Price')
plt.plot(np.arange(time_steps, len(train_predict)+time_steps), train_predict[:,0], label='Predicted Train Price')
plt.plot(np.arange(len(y_train[0]), len(y_train[0])+len(y_test[0])), y_test[0], label='Actual Test Price')
plt.plot(np.arange(len(y_train[0])+time_steps, len(y_train[0])+len(test_predict)+time_steps), test_predict[:,0], label='Predicted Test Price')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()

In [126]:
# Save the model
model.save("static/model/lstm_model.h5")
print("Model saved successfully.")

Model saved successfully.


## TEST PREDIKSI UNTUK 7 HARI KEDEPAN

In [133]:
# Load Model
my_model = load_model("static/model/lstm_model.h5")

In [173]:
# Create input for prediction
last_30_days = dataset[:30]
X_input = np.reshape(last_30_days, (1, 30, 1))

In [175]:
# Predict for next 7 days
predicted_prices = []
for _ in range(7):
    next_day_prediction = model.predict(X_input)
    predicted_prices.append(next_day_prediction[0, 0])
    X_input = np.append(X_input[:,1:,:], next_day_prediction.reshape(1,1,1), axis=1)


1/1 [==============================] - 0s 36ms/step


In [176]:
# Inverse transform predictions
print(predicted_prices)
predicted_prices = scaler.inverse_transform(np.array(predicted_prices).reshape(-1, 1))

[0.4109317, 0.27892196, 0.26679745, 0.34850767, 0.3269497, 0.2606398, 0.20985527]


In [2]:
# Print predicted prices
print("Predicted prices for the next 7 days:")
for i in range(7):
    tanggal_prediksi = konversi_tanggal(i+1)
    print(f"Day {tanggal_prediksi} : ${predicted_prices[i][0]:.2f}")

Predicted prices for the next 7 days:


NameError: name 'konversi_tanggal' is not defined